# Document Similarity using NLTK and gensim

### tokenization, word counts, and possibly calculated tf-idf scores for words

### Tokenization in NLTK

In [ ]:
import nltk
import string

from collections import Counter
import urllib

# nltk 코퍼스 파일 받아오고 열기
text = nltk.corpus.gutenberg.raw("burgess-busterbrown.txt")

# raw text를 모두 소문자로 바꾸고 특수문자를 제거한 뒤 tokenize하기
def get_tokens(text):
    lowers = text.lower()
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    return tokens

tokens = get_tokens(text)
print(tokens)


In [ ]:
# tokenize된 문서 안에서 각 단어의 빈도 세기
count = Counter(tokens)

# 가장 많이 쓰인 단어부터 상위 10개 확인
print(count.most_common(10))

In [ ]:
# nltk를 이용해 stopword 제거하기 
from nltk.corpus import stopwords

tokens = get_tokens(text)
filtered = [w for w in tokens if not w in stopwords.words('english')]

# stopword가 제거된 토큰들의 빈도 세기
count = Counter(filtered)

# 가장 많이 쓰인 토큰부터 상위 100개 확인
print(count.most_common(100))

In [ ]:
# 각 토큰의 stem 얻기

from nltk.stem.porter import *

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)

# stemmize된 토큰들의 빈도 세기
count = Counter(stemmed)

# 가장 많이 쓰인 stem부터 상위 100개 확인
print(count.most_common(100))

### Tf-Idf in gensim
###### gensim 설치 명령: pip install --upgrade gensim
###### gensim installation instructions: https://radimrehurek.com/gensim/install.html

In [ ]:
import nltk
import string
import gensim

# 지금까지 했던 tokenize, stopword 제거, punctuation 제거, stemmize를 모두 수행하는 새 tokenize 함수
def tokenize(text):
    tokens = get_tokens(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

# nltk gutenberg 텍스트의 목록 불러오기
titles = nltk.corpus.gutenberg.fileids()
print(titles)

# 목록에 있는 텍스트를 순서대로 tokenize(stemize)한 뒤 하나의 리스트로 모으기
text_tokens = []
for title in titles:
    text = nltk.corpus.gutenberg.raw(title)
    tokens = tokenize(text)
    text_tokens.append(tokens)
    

In [ ]:
# 각 리스트에 들어 있는 tokenized text의 형태
print(text_tokens[0])

In [ ]:
# gensim 패키지를 이용하여 단어들을 숫자 인덱스로 바꾸어 쓰기
# 딕셔너리 형태로 저장

from gensim import corpora
dictionary = corpora.Dictionary(text_tokens)

# 인덱스로 단어 호출하기 
print(dictionary[10])
print(dictionary[100])

# 단어로 인덱스 호출하기 
print(dictionary.token2id['road'])
print(dictionary.token2id['like'])

In [ ]:
# 딕셔너리 내 단어들의 총 개수 
print("Number of words in dictionary:",len(dictionary))

# 각 인덱스가 부여된 단어들의 목록 
for i in dictionary.keys():
    print(i, dictionary[i])

In [ ]:
# 위에서 정의한 인덱스에 따라 각 텍스트를 bag of words의 형태로 바꾸어 코퍼스를 구성
corpus = [dictionary.doc2bow(text) for text in text_tokens]

print(dictionary.doc2bow(text_tokens[0]))

In [ ]:
# create a tf-idf model from the corpus
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)

# tf-idf 모델에 따라 문서에 사용된 단어들의 값이 산출됨
tf_idf[corpus[0]]

In [ ]:
# a similarity measure object in tf-idf space
sims = gensim.similarities.Similarity('', tf_idf[corpus], num_features=len(dictionary))
print(type(sims))

# Each shard is stored to disk under output_prefix.shard_number.
# If you don’t specify an output prefix, a random filename in temp will be used.

In [ ]:
print(sims[tf_idf[corpus[4]]])

In [ ]:
# create a query document and convert it to tf-idf

# query document을 위의 방법에 따라 tokenize 하기
query_doc = [w.lower() for w in tokenize("Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence.")]
print(query_doc)

In [ ]:
# query document를 위 코퍼스의 인덱스에 따라 bag of words로 변환
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)

In [ ]:
# query document를 위 tf-idf 모델에 따라 표현
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

In [ ]:
# document similarities to query
print(sims[query_doc_tf_idf])